In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy

In [2]:
# Import Username, Passwrd and Database Name 
from config import pg_ipaddress, pg_port, pg_username, pg_password, pg_dbname

In [3]:
# Setup connection with Postgres
from sqlalchemy import create_engine
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=pg_username,
                       password=pg_password,
                       ipaddress=pg_ipaddress,
                       port=pg_port,
                       dbname=pg_dbname))

connection = create_engine(postgres_str)

In [ ]:
employees_df = pd.read_sql("select * from employees", connection)
employees_df.fillna("NaN")

In [ ]:
# Create a histogram to visualize the most common salary ranges for employees
salaries_df = pd.read_sql("select * from salaries", connection)
salaries_df.fillna("NaN")

In [ ]:
salaries_df["salary"].min()

In [ ]:
salaries_df["salary"].max()

In [ ]:
plt.figure(figsize=(10,7))
bins = [40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000]
n, bins, patches = plt.hist(x=salaries_df["salary"], bins = bins, color = 'g', alpha=0.7, rwidth=0.9)
plt.grid(axis='y', alpha=0.7)
plt.axvline(salaries_df['salary'].median(), color='b', linestyle='dashed', linewidth=2, label="Median Salary")
plt.xlim(40000, 130000)
plt.ylim(0, 180000)
plt.xlabel("Salary($)")
plt.ylabel("Number of Employees")
plt.legend() 
plt.title("Most Common Salary Ranges for Pewlett Hackard Employees")
plt.savefig("Common_Salary_Ranges.png")

In [ ]:
# Create bar chart of average salary by title
titles_df = pd.read_sql("select * from titles", connection)

# Rename title_id to emp_title_id
titles_df = titles_df.rename(columns = {"title_id": "emp_title_id"})

titles_df.fillna("NaN")

In [ ]:
emp_salary_df = pd.merge(employees_df, salaries_df, on = "emp_no")
emp_salary_df

In [ ]:
avg_salary_by_title = emp_salary_df.groupby(["emp_title_id"])["salary"].mean()
avg_salary_by_title

In [ ]:
salary_by_title_df = pd.merge(titles_df, avg_salary_by_title, on = "emp_title_id")
salary_by_title_df

In [ ]:
plt.figure(figsize=(10,7))
x_axis = np.arange(len(salary_by_title_df))
ticks = [value for value in x_axis]
salary_by_title_df = salary_by_title_df.sort_values("salary", ascending = False)
chart=plt.bar(x_axis, salary_by_title_df["salary"], color = 'b', alpha = 0.7, align="center")
plt.grid(axis='y', alpha=0.7)
plt.xticks(ticks, salary_by_title_df["title"], rotation="45", horizontalalignment = "right")
plt.xlim(-0.7, len(x_axis))
plt.ylim(0, 70000)
plt.title('Average Salary by Title for Pewlett Hackard Employees', fontsize = 14)
plt.xlabel('Title')
plt.ylabel('Average Salary($)')
for i in chart:
    yvalue = i.get_height()
    plt.text(i.get_x(), yvalue + 500, "${:,.2f}K".format(yvalue/1000))
plt.savefig('Average_Salary_by_Title')
plt.show()